In [123]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns 

df = pd.read_csv('StockData.csv')
df.head()

,DATE,OP,MAXP,MINP,CLP,PERC,VOL,MA5,MA10,MA20,...,ZIG5,ZIGTRN,KDHL,KDCRS,OSC,OSCUD,OSCTRN,BBANDU,BBANDD,BBANDTCH
0,20131021,111.0,111.5,109.5,110.0,0.0000,41327.324,108.0,106.45,104.83,...,110.0,1.0,4.06,NaN,0.50,NaN,NaN,NaN,NaN,NaN
1,20131022,110.0,111.5,109.5,111.5,1.3636,39837.172,108.9,107.30,105.13,...,111.5,-1.0,9.38,0.0,0.62,1.0,0.0,NaN,NaN,NaN
2,20131023,111.0,111.5,110.0,111.0,-0.4484,23057.730,109.9,107.90,105.43,...,111.0,0.0,10.35,0.0,0.61,-1.0,-1.0,NaN,NaN,NaN
3,20131024,109.0,110.0,108.5,110.0,-0.9009,21570.336,110.5,108.40,105.78,...,110.5,0.0,5.88,0.0,0.50,-1.0,0.0,NaN,NaN,NaN
4,20131025,108.5,109.0,106.5,107.0,-2.7273,37244.500,109.9,108.45,106.05,...,110.0,0.0,0.00,-1.0,0.19,-1.0,0.0,NaN,NaN,NaN


In [124]:
X = df.loc[:, ['OP', 'MAXP', 'MINP', 'CLP', 'PERC', 'VOL', 'MA5', 'MA10', 'MA20', 'MA60', 'BIAS5', 'BIAS10', 'ZIG5', 'ZIGTRN', 'KDHL', 'KDCRS', 'OSC', 'OSCTRN', 'BBANDTCH']][20:-1]
y = df.CLP[20:]

In [125]:
window_size = 10
roll_qty = 1

RX = np.array(np.expand_dims(X[:window_size], axis=0))
Ry = np.array(y[window_size:])

for i in range(window_size + 1, len(X) + 1, roll_qty):
    RX = np.concatenate([RX, np.expand_dims(X[i - window_size: i], axis=0)], axis=0)

print(RX.shape, Ry.shape)

(2419, 10, 19) (2419,)


In [126]:
train_size = int(len(Ry) * 0.7 * 0.7)
val_size = int(len(Ry) * 0.7 * 0.3)

X_train = RX[:train_size]
y_train = Ry[:train_size]

X_val = RX[train_size:train_size + val_size]
y_val = Ry[train_size:train_size + val_size]

X_test = RX[train_size + val_size:]
y_test = Ry[train_size + val_size:]

In [136]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(10, 19)),
    tf.keras.layers.Dense(700)
])

model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['mean_squared_error'])

In [138]:
history = model.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=[X_val, y_val])

Epoch 1/20
119/119 [==============================] - 1s 6ms/step - loss: 4.8666 - mean_squared_error: 33129.3750 - val_loss: 9.0724 - val_mean_squared_error: 93886.1250
Epoch 2/20
119/119 [==============================] - 1s 6ms/step - loss: 4.8655 - mean_squared_error: 33191.3242 - val_loss: 9.1775 - val_mean_squared_error: 93967.3047
Epoch 3/20
119/119 [==============================] - 1s 6ms/step - loss: 4.8619 - mean_squared_error: 33237.0898 - val_loss: 9.2590 - val_mean_squared_error: 94040.2656
Epoch 4/20
119/119 [==============================] - 1s 6ms/step - loss: 4.8591 - mean_squared_error: 33278.1914 - val_loss: 9.3353 - val_mean_squared_error: 94107.9688
Epoch 5/20
119/119 [==============================] - 1s 6ms/step - loss: 4.8584 - mean_squared_error: 33320.6328 - val_loss: 9.3745 - val_mean_squared_error: 94212.7188
Epoch 6/20
119/119 [==============================] - 1s 6ms/step - loss: 4.8654 - mean_squared_error: 33403.4102 - val_loss: 9.5154 - val_mean_square

In [145]:
y_pred = model.predict(X_test)
y_pred.shape
# plt.plot()
# mat = confusion_matrix(y_test, y_pred)
# plot_confusion_matrix(conf_mat=mat, figsize=(4, 4), show_normed=True)

23/23 [==============================] - 0s 3ms/step


(727, 700)